In [44]:
import matplotlib.pyplot as plt
import numpy as np
import math

# f = open('data/retrace.csv')
# tokens = [e.split(",") for e in f.readlines()]
# x = [int(e[0]) for e in tokens]
# y = [int(e[1]) for e in tokens]
# handle = plt.plot(x,y)

def get_trace_file(filename, I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    #maxy = max([e[1] for e in tokens])
    x = [e[0] for e in tokens]
    y = [e[1] for e in tokens]
    return (x,y)

def get_detect_file(filename,I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    x = [e[0] for e in tokens]
    y = [e[1] for e in tokens]
    c = ['r' if e[2] == 1 else ('g' if e[2] == 0 else 'b') for e in tokens]
    return (x,y,c)

def get_event_file(filename,I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    tokens = [e for e in tokens if e[2] == e[2]]
    tokens = [e for e in tokens if e[2] > 0]
    #tokens = [e for e in tokens if abs(e[2]) > 2]
    return tokens


In [45]:
def create_trace_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    plt.ylim(J)
    plt.plot(data[0],data[1])

def plot_gaussian(datum):
    mu = datum[0]
    A = datum[1]
    sigma = datum[2]
    x = np.arange(mu - 100, mu + 100, 0.1)
    y = A*np.exp(-0.5*np.square(x - mu)/(sigma*sigma))
    plt.plot(x, y, linestyle = "dashed")

def create_event_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    plt.ylim(J)
    plt.subplot(ax)
    for datum in data:
        plot_gaussian(datum)
    #plt.scatter(x1,y,marker='|',color='r')
    #plt.scatter(x,y,marker='.',color='g')
    #plt.scatter(x2,y,marker='|',color='b')

def create_plot(I):
    data = [get_trace_file('Saves/output_baselined.csv',I),
            get_trace_file('Saves/output_smoothed.csv',I),
            get_trace_file('Saves/output_diff.csv',I),
            get_trace_file('Saves/output_cuts.csv',I),
            get_trace_file('Saves/output_simulated.csv',I),
            get_event_file('Saves/output_pulses.csv',I)
    ]
    fig = plt.figure(figsize = (14,12))
    maxy = max(data[0][1]) + 10
    ax = plt.subplot(2,1,1)
    create_trace_plot(ax,data[0],I,(0,maxy))
    create_trace_plot(ax,data[1],I,(0,maxy))
    create_trace_plot(ax,data[2],I,(0,maxy))
    #create_trace_plot(ax,data[2],I,(0,maxy))
    #create_event_plot(ax,data[3],I,(0,maxy))

    return fig,ax



In [46]:
from ipywidgets import interact, widgets

def func(delta, size):
    I = (delta,delta + size)
    fig,ax = create_plot(I)

interact(func,
    delta = widgets.IntSlider(min=0, max=30000, step=100, value=5000),
    size = widgets.IntSlider(min=100, max=25000, step=100, value=1000)
)

interactive(children=(IntSlider(value=5000, description='delta', max=30000, step=100), IntSlider(value=1000, d…

<function __main__.func(delta, size)>

\begin{align*}
f(x) &= A\exp(-\frac{(x - \mu)^2}{2\sigma^2}) + B
\\
\ln(f(x) - B) &= \ln(A) - \frac{(x - \mu)^2}{2\sigma^2}
\\
f'(x) &= - \frac{x - \mu}{\sigma^2}(f(x) - B)
\\
f''(x) &= - \frac{1}{\sigma^2}(f(x) - B) - \frac{x - \mu}{\sigma^2}f'(x)
\\
 &= - \frac{1}{\sigma^2}(f(x) - B) + \frac{(x - \mu)^2}{\sigma^4}(f(x) - B)
\\
 &= (\frac{(x - \mu)^2}{\sigma^2} - 1)\frac{f(x) - B}{\sigma^2}
\end{align*}


Let $x_i,y_i,y_i'$ be defined for $i = 1,2$ such that $f(x_i) = y_i, f'(x_i) = y_i'$, then:
\begin{align*}
\ln(y_i - B) &= \ln(A) - \frac{(x_i - \mu)^2}{2\sigma^2}
\\
\frac{y'_i}{y_i - B} &= - \frac{x_i - \mu}{\sigma^2}
\end{align*}
So we have:
\begin{align*}
\ln(y_1 - B) &= \ln(A) - \frac{(x_1 - \mu)^2}{2\sigma^2}
\\
\frac{y'_1}{y_1 - B} &= - \frac{x_1 - \mu}{\sigma^2}
\\
\ln(y_2 - B) &= \ln(A) - \frac{(x_2 - \mu)^2}{2\sigma^2}
\\
\frac{y'_2}{y_2 - B} &= - \frac{x_2 - \mu}{\sigma^2}
\end{align*}

These can be combined to get
\begin{align*}
\ln(y_1 - B) &= \ln(A) - \frac{y'_1}{y_1 - B}\frac{x_1 - \mu}{2}
\\
\ln(y_2 - B) &= \ln(A) - \frac{y'_2}{y_2 - B}\frac{x_2 - \mu}{2}
\\
\ln(y_1 - B) - \ln(y_2 - B) &= \frac{y'_2}{y_2 - B}\frac{x_2 - \mu}{2} - \frac{y'_1}{y_1 - B}\frac{x_1 - \mu}{2}
\end{align*}